In [1]:
import numpy as np
import torch
import torch.nn as nn
from time import time
from tqdm import tqdm

In [2]:
class inference_dataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        # convert from numpy to tensor
        x = torch.tensor(self.data[index], dtype=torch.float32)

        return x

In [3]:
# MIMENet Deep Neural Network
class MIMENetEnsemble(nn.Module):
    # Constructor for 5 fully connected layers with bottleneck for layers 4 and 5
    # dropout after each layer
    def __init__(self, input_size, hidden_size_factor, bottleneck, output_size):
        super(MIMENetEnsemble, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size_factor*input_size)
        self.dropout1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(hidden_size_factor*input_size, hidden_size_factor*input_size)
        self.dropout2 = nn.Dropout(p=0.2)
        self.fc3 = nn.Linear(hidden_size_factor*input_size, hidden_size_factor*input_size)
        self.dropout3 = nn.Dropout(p=0.2)
        self.fc4 = nn.Linear(hidden_size_factor*input_size, int(hidden_size_factor*input_size*bottleneck))
        self.dropout4 = nn.Dropout(p=0.2)
        self.fc5 = nn.Linear(int(hidden_size_factor*input_size*bottleneck), output_size)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    # Forward pass
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.relu(self.fc4(x))
        x = self.dropout4(x)
        x = self.sigmoid(self.fc5(x))

        return x

In [4]:
def predict(model, x, n, batch_size):
    """This function is used for inference using the model. It takes in a 2d tensor
    and predicts the output using the model. For uncertainty quantification, the
    model is run n times and the full distribution is returned.

    Args:
        x (tensor): Input tensor
        n (int): Number of times to run the model
    """
    # set length of input
    length = x.shape[0]

    # set device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    
    # define x as inference dataset
    x = inference_dataset(x)

    # define dataloader
    dataloader = torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=False)

    # set model
    model = model.to(device)

    # initialize output
    output = np.zeros((length, n))
    # run model n times
    for i in tqdm(range(n)):
        for j, data in enumerate(dataloader):
            # only send data to device
            data = data.to(device)

            # add to output
            output[j*batch_size:(j+1)*batch_size, i] = model(data).squeeze().cpu().detach().numpy()
            
    return output

In [5]:
# test predict function
model = MIMENetEnsemble(535*4+8, 2, 0.5, 1)
batch_size = 2**14
# 535*4 by 535*4+8 tensor random input
x = np.random.rand(535*3, 535*4+8)

start = time()
predictions = predict(model, x, 1000, batch_size)
end = time()
print(end-start)
print(predictions.shape)

cuda:0


100%|██████████| 1000/1000 [00:27<00:00, 35.97it/s]

28.6442608833313
(1605, 1000)


In [6]:
# test predict function
model = MIMENetEnsemble(535*4+8, 2, 0.5, 1)
batch_size = 2**14
# 535*4 by 535*4+8 tensor random input
x = np.random.rand(1285605, 535*4+8)

start = time()
predictions = predict(model, x, 1000, batch_size)
end = time()
print(end-start)
print(predictions.shape)

cuda:0


  0%|          | 3/1000 [01:18<7:17:10, 26.31s/it]


KeyboardInterrupt: 